In [1]:
import pandas as pd
from models import base_res_net
import tensorflow as tf
from tensorflow import keras
import tensorflow.compat.v1 as tfc

from src import InputPipeline

%load_ext autoreload
%autoreload 2

## Check GPU for tf

In [2]:
# Some GPU setup
# for documentation about using gpus referto: https://www.tensorflow.org/install/pip#windows-wsl2

tf.keras.backend.clear_session()

device_name = tf.test.gpu_device_name()
if not device_name:
  raise SystemError('GPU device not found')
try:  # prevent a previous session from being alive
  sess.close() 
except:
  pass

tfc.enable_eager_execution()
gpu_options= tfc.GPUOptions(per_process_gpu_memory_fraction = 0.8)
sess = tfc.InteractiveSession(config=tfc.ConfigProto(gpu_options=gpu_options))

In [3]:
train_df = pd.read_csv("data/train_images.csv")

## Configure Model

In [55]:
INPUT_SHAPE = (218,218,3)

CONF = {
  "learning_rate": 0.001,
  "batch_size": 32,
  "epochs": 50,
  "input_width": INPUT_SHAPE[0],
  "input_height": INPUT_SHAPE[1],
  "loss_function": "sparse_categorical_crossentropy",
}

In [56]:
base_model = base_res_net(INPUT_SHAPE, len(train_df["label"].unique()))
base_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 218, 218, 3  0           []                               
                                )]                                                                
                                                                                                  
 sequential_2 (Sequential)      (None, 218, 218, 3)  0           ['input_3[0][0]']                
                                                                                                  
 conv2d_10 (Conv2D)             (None, 109, 109, 16  448         ['sequential_2[0][0]']           
                                )                                                                 
                                                                                            

## Make Input Pipeline

In [57]:
input_pipeline = InputPipeline(splits=(0.8,0.0,0.2), channels=3, batch_size=CONF["batch_size"], size=INPUT_SHAPE[:2])
input_pipeline.make_train_datasets(directory="data/train_images/train_images")

Found 3926 files belonging to 200 classes.
Using 3141 files for training.
Using 785 files for validation.
Datasets populated!


## Modeling

In [58]:
cached_train, cached_val = input_pipeline.get_cached_train_datasets()

In [59]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
base_model.compile(loss=CONF["loss_function"],
             optimizer=keras.optimizers.Adam(CONF["learning_rate"]),
             metrics=['categorical_accuracy'],
            )

In [ ]:
base_model.fit(cached_train, epochs=CONF["epochs"], validation_data=cached_val,callbacks=[callback],)

Epoch 1/50
99/99 [==============================] - 43s 364ms/step - loss: 21.4838 - categorical_accuracy: 0.0115 - val_loss: 6.6386 - val_categorical_accuracy: 0.0000e+00
Epoch 2/50
99/99 [==============================] - 31s 311ms/step - loss: 15.1652 - categorical_accuracy: 0.0280 - val_loss: 11.0211 - val_categorical_accuracy: 0.0000e+00
Epoch 3/50
99/99 [==============================] - 31s 318ms/step - loss: 14.7179 - categorical_accuracy: 0.0111 - val_loss: 11.6089 - val_categorical_accuracy: 0.0000e+00
Epoch 4/50
61/99 [=================>............] - ETA: 11s - loss: 12.6257 - categorical_accuracy: 0.0118

In [14]:
input_pipeline.make_test_dataset("data/test_images")
test_ds = input_pipeline.get_cached_test_datasets()

Found 4000 files belonging to 1 classes.


In [17]:
pred = base_model.predict(test_ds)

42/42 [==============================] - 3s 61ms/step


In [45]:
pred_df=pd.DataFrame(pred)
test_df = pd.read_csv("data/test_images_sample.csv", index_col="id")
test_df["label"] = list(pred_df.idxmax(axis=1))

In [46]:
test_df.to_csv("data/test_images_sample.csv")